# Preprocessing of images for InceptionV3 pretrained model
This version 2 includes the fix of an error based on implemented preprocessing of pictures (1/255)

In [36]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

In [37]:
INPUT_PATH = "../input/flickr8k/"
IMAGE_PATH = INPUT_PATH+'Images/'
OUTPUT_PATH = "./"
OUTPUT_IMAGE_PATH = OUTPUT_PATH +'Images/'
CAPTIONS_FILE = INPUT_PATH+'captions.txt'

In [38]:
# Create a dataframe which summarizes the image, path & captions as a dataframe
# Each image id has 5 captions associated with it therefore the total dataset should have 40455 samples.

captions_df = pd.read_csv(CAPTIONS_FILE)

pd.set_option('display.max_colwidth',-1) # Set the max column width to see the complete caption
print(captions_df.shape)
captions_df.head()

(40455, 2)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a wooden cabin .


In [39]:
captions_agg_df=captions_df.groupby("image").first().reset_index()
captions_agg_df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
2,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .
3,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
4,1007129816_e794419615.jpg,A man in an orange hat starring at something .


In [40]:
datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=captions_agg_df, 
                                            directory=IMAGE_PATH, 
                                            x_col="image", y_col="caption", 
                                            class_mode="raw", 
                                            target_size=(299,299), 
                                            shuffle=False
                                            #batch_size=1024
                                           )


Found 8091 validated image filenames.


In [41]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [46]:
!mkdir $OUTPUT_IMAGE_PATH

In [47]:
i=0

for X, _ in train_generator:
    
    img = tf.keras.applications.inception_v3.preprocess_input(X)    
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))
    if i == 0:
        print(f"X.shape: {X.shape}")
        print(f"img.shape: {img.shape}")
        print(f"batch_features.shape: {batch_features.shape}")
        print(f"reshaped batch_features.shape: {batch_features.shape}")

    for bf in batch_features:
        filename = OUTPUT_IMAGE_PATH + captions_agg_df.iloc[i].image.replace(".jpg",".npy")
        print(f"filename: {filename}")
        np.save(filename, bf.numpy())
        i=i+1


X.shape: (32, 299, 299, 3)
img.shape: (32, 299, 299, 3)
batch_features.shape: (32, 8, 8, 2048)
reshaped batch_features.shape: (32, 64, 2048)
filename: ./Images/1000268201_693b08cb0e.npy
filename: ./Images/1001773457_577c3a7d70.npy
filename: ./Images/1002674143_1b742ab4b8.npy
filename: ./Images/1003163366_44323f5815.npy
filename: ./Images/1007129816_e794419615.npy
filename: ./Images/1007320043_627395c3d8.npy
filename: ./Images/1009434119_febe49276a.npy
filename: ./Images/1012212859_01547e3f17.npy
filename: ./Images/1015118661_980735411b.npy
filename: ./Images/1015584366_dfcec3c85a.npy
filename: ./Images/101654506_8eb26cfb60.npy
filename: ./Images/101669240_b2d3e7f17b.npy
filename: ./Images/1016887272_03199f49c4.npy
filename: ./Images/1019077836_6fc9b15408.npy
filename: ./Images/1019604187_d087bf9a5f.npy
filename: ./Images/1020651753_06077ec457.npy
filename: ./Images/1022454332_6af2c1449a.npy
filename: ./Images/1022454428_b6b660a67b.npy
filename: ./Images/1022975728_75515238d8.npy
filena

IndexError: single positional indexer is out-of-bounds